# Data generation for segmentation model training

## Importing libraries

In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../')  # This adds the parent directory to the path

## Background substraction

In [6]:
from src.mask_extractor import MaskExtractor

mask_extractor = MaskExtractor(
    images_dir = '../Datasets/OakGait16/segmentation_frames/',
    clips_directory = '/mnt/c/Users/chris/Documents/OAKGait16/clips/', #Where the clips are saved in your computer
    mks_dir = 'masks/',
    frs_dir = 'rgb/',
    subjects = ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016'],
    views = ['060', '075', '090', '105', '120'],
    nclips = {'nm':5, 'bg':5, 'cl':5},
    verbose = True,
)

mask_extractor.extract_masks()

PROCESSING VIEW: 060
Processing subject: 000 view: 060...
Processing subject: 001 view: 060...
Processing subject: 002 view: 060...
Processing subject: 003 view: 060...
Processing subject: 004 view: 060...
Processing subject: 005 view: 060...
Processing subject: 006 view: 060...
Processing subject: 007 view: 060...
Processing subject: 008 view: 060...
Processing subject: 009 view: 060...
Processing subject: 010 view: 060...
Processing subject: 011 view: 060...
Processing subject: 012 view: 060...
Processing subject: 013 view: 060...
Processing subject: 014 view: 060...
Processing subject: 015 view: 060...
Processing subject: 016 view: 060...
PROCESSING VIEW: 075
Processing subject: 000 view: 075...
Processing subject: 001 view: 075...
Processing subject: 002 view: 075...
Processing subject: 003 view: 075...
Processing subject: 004 view: 075...
Processing subject: 005 view: 075...
Processing subject: 006 view: 075...
Processing subject: 007 view: 075...
Processing subject: 008 view: 075

## Region of interest & Ground truth obtention

In [7]:
ie = Core()
device = "CPU"
model = ie.read_model(model="/home/christian/Documents/CNN-Gait-Recognition-Offline/models/public/mobilenet-ssd/FP32/mobilenet-ssd.xml")
compiled_ssd = ie.compile_model(model=model, device_name=device)
input_layer_ir = next(iter(compiled_ssd.inputs))
output_layer_ir = next(iter(compiled_ssd.outputs))



def find_ROI(img):
        img_orig = np.copy(img)
        

        img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_RGB2BGR),(300,300))
        input_image = cv2.dnn.blobFromImage(img, size=(300, 300), ddepth=cv2.CV_8U)

        input_image = reshape(input_image, [-1,3,300,300])
        result = compiled_ssd.infer_new_request({0: input_image})

        x_min = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),3]*img_orig.shape[1]
        y_min = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),4]*img_orig.shape[0]
        x_max = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),5]*img_orig.shape[1]
        y_max = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),6]*img_orig.shape[0]


        (x, y, w, h) = int(x_min), int(y_min), int(x_max), int(y_max)

        offsets = [10, 10, 10, 10]
        x = x - offsets[0] if x - offsets[0] >=0 else 0
        y = y - offsets[3] if y - offsets[3] >=0 else 0
        w = w + offsets[1]+offsets[0] if w + offsets[1]+offsets[0] <=img_orig.shape[1] else img_orig.shape[1]
        h = h + offsets[3]+offsets[1] if h + offsets[3]+offsets[1] <=img_orig.shape[0] else img_orig.shape[0]
        
        return x, y, w, h

In [8]:
images_dir = '/home/christian/Documents/Datasets/OakGait16/frames/'
rois_dir = '/home/christian/Documents/Datasets/OakGait16/rois/'
gt_rois_dir = '/home/christian/Documents/Datasets/OakGait16/gt_rois/'
frs_dir = 'rgb_segmentation/'
mks_dir = 'masks/'
views = ['060','075','090','105','120']
training_silhouettes = '/home/christian/Documents/Datasets/OakGait16/rgb_silhouettes/'
masked_silhouettes = '/home/christian/Documents/Datasets/OakGait16/masked_silhouettes/'

In [11]:
create_dir(training_silhouettes, force=True)
create_dir(masked_silhouettes, force=True)
c = 0
for view in views:
    frames_dir = os.path.join(images_dir, view, frs_dir)
    masks_dir = os.path.join(images_dir, view, mks_dir)
    rep_dir = os.path.join(rois_dir, view)
    gt_dir = os.path.join(gt_rois_dir, view)
    print(f'GENERATING GAIT REPRESENTATIONS FROM VIEW: {view}')
    subjects = sorted(os.listdir(os.path.join(images_dir, view, frs_dir)))
    
    for subject in subjects:
#     for subject in ['022', '023', '025']:
        print(f'Processing subject: {subject} view: {view}')
        

        for j, walk in enumerate(['nm','bg','cl']):
#         for j, walk in enumerate(['nm']):
            seq_dir = os.path.join(frames_dir, subject, walk)
            seqs = sorted(os.listdir(seq_dir))
            seqs = np.random.choice(seqs, size=2, replace=False)
            print(f'Processing: {seqs}')
            for s, seq in enumerate(seqs):

                seq_frames_dir = os.path.join(seq_dir, seq)
                lfiles = sorted(os.listdir(seq_frames_dir))
                cnt = 0
                for idimg, path in enumerate(lfiles):
                    
                    
                        img = cv2.imread(os.path.join(seq_frames_dir, path))
                        mask = cv2.imread(os.path.join(masks_dir, subject, walk, seq, path),0)


                        try: 
                            x,y,w,h = find_ROI(img)
                            bbox =  [x, y, w, h]
                            gti = mask[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                            roi = img[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                            
                            gti_path = os.path.join(masked_silhouettes, f'{subject}-{seq}-{str(c).zfill(7)}.jpg')
                            roi_path = os.path.join(training_silhouettes, f'{subject}-{seq}-{str(c).zfill(7)}.jpg')

                            if roi.shape[1] > 60  or roi.shape[1] < 150:
                                cv2.imwrite(gti_path, gti)
                                cv2.imwrite(roi_path, roi)
                                c += 1
                            
                        except:
                            pass           

GENERATING GAIT REPRESENTATIONS FROM VIEW: 060
Processing subject: 000 view: 060
Processing: ['nm-03' 'nm-02']


2022-09-18 07:37:34.615990: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-09-18 07:37:35.754648: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-18 07:37:35.805219: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 940MX computeCapability: 5.0
coreClock: 1.2415GHz coreCount: 3 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 14.30GiB/s
2022-09-18 07:37:35.805334: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-09-18 07:37:36.417388: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-09-18 07:37:36.417574: I tensorflow/stream_executor/platfor

Processing: ['bg-03' 'bg-04']
Processing: ['cl-04' 'cl-02']
Processing subject: 001 view: 060
Processing: ['nm-02' 'nm-05']
Processing: ['bg-04' 'bg-02']
Processing: ['cl-05' 'cl-01']
Processing subject: 002 view: 060
Processing: ['nm-04' 'nm-01']
Processing: ['bg-01' 'bg-03']
Processing: ['cl-02' 'cl-04']
Processing subject: 003 view: 060
Processing: ['nm-01' 'nm-04']
Processing: ['bg-02' 'bg-05']
Processing: ['cl-01' 'cl-04']
Processing subject: 004 view: 060
Processing: ['nm-04' 'nm-02']
Processing: ['bg-05' 'bg-03']
Processing: ['cl-04' 'cl-01']
Processing subject: 005 view: 060
Processing: ['nm-04' 'nm-03']
Processing: ['bg-01' 'bg-04']
Processing: ['cl-04' 'cl-05']
Processing subject: 006 view: 060
Processing: ['nm-02' 'nm-03']
Processing: ['bg-04' 'bg-02']
Processing: ['cl-03' 'cl-01']
Processing subject: 007 view: 060
Processing: ['nm-01' 'nm-02']
Processing: ['bg-01' 'bg-05']
Processing: ['cl-02' 'cl-01']
Processing subject: 008 view: 060
Processing: ['nm-04' 'nm-03']
Processi